# Introduction

This notebook shows how to work with the Weaviate Streamlit connector. 

The Weaviate Streamlit connector is a Python package that allows you to easily create Streamlit apps that loads data from a Weaviate instance.

# Imports

In [1]:
from st_weaviate_connection import WeaviateConnection
import streamlit as st
import os

# Usage

## Connect to a Weaviate Cloud instance

### Requirements

#### Weaviate Cloud instance

The easiest way to use this connector is with a [Weaviate Cloud](https://console.weaviate.cloud) instance, and to connect using the URL and the API key.

This demo uses the following read-only credentials:

```python
weaviate_url = "https://hha2nvjsruetknc5vxwrwa.c0.europe-west2.gcp.weaviate.cloud"
weaviate_apikey = "nMZuw1z1zVtnjkXXOMGx9Ows7YWGsakItdus"
```

You can create a free Weaviate Cloud instance by signing up [here](https://console.weaviate.cloud).

#### (Optional) Inference API key

The demo notebook uses hybrid search, which combines semantic search with a traditional (keyword) search to provide more accurate results. 

The `Movie` collection is set up to use Cohere for semantic search. 

- If you do not have an Cohere API key, you can sign up on their website.
- If you do not wish to use the semantic search part of hybrid search, set the `alpha` value to `0` in the `hybrid_search` function.

In [2]:
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_apikey = os.environ["WEAVIATE_API_KEY"]
cohere_apikey = os.environ["COHERE_API_KEY"]  # Optional (for semantic search)

conn = st.connection(
    "weaviate",
    type=WeaviateConnection,
    url=weaviate_url,
    api_key=weaviate_apikey,
    additional_headers={"X-Cohere-Api-Key": cohere_apikey},
)

2024-07-30 21:56:44.650 
  command:

    streamlit run /Users/jphwang/Library/Caches/pypoetry/virtualenvs/st-weaviate-connection-6uLZqwja-py3.11/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


## Querying Data

There are three convenience functions in the connector to query data:

- `.hybrid_search`: Perform a hybrid search, which is a weighted combination of a semantic search and a keyword search.
- `.graphql_query`: Perform a raw GraphQL query

### Basic hybrid search

A hybrid search is a weighted combination of a semantic search and a keyword search.

For basic hybrid search, just provide the collection name and the search term.


In [3]:
df = conn.hybrid_search(
    collection_name="MovieDemo",
    query="Fantasy or sci-fi drama",
)

df.head()

,movie_id,vote_count,budget,tagline,title,revenue,overview,release_year,vote_average,genres
0,83542,6786,102000000,Everything is Connected,Cloud Atlas,130482868,A set of six nested stories spanning time betw...,2012,6.882,"[Drama, Science Fiction]"
1,1124,15086,40000000,Are You Watching Closely?,The Prestige,109676311,A mysterious story of two magicians whose inte...,2006,8.203,"[Drama, Mystery, Science Fiction]"
2,419704,6271,87500000,The answers we seek are just outside our reach,Ad Astra,127461872,"The near future, a time when both hope and har...",2019,6.100,"[Science Fiction, Drama]"
3,9294,1118,32000000,Some things in life just can't be explained.,Phenomenon,152000000,An ordinary man sees a bright light descend fr...,1996,6.402,"[Drama, Romance, Science Fiction, Fantasy]"
4,58244,2431,60000000,Two worlds. One future.,Upside Down,22187813,In an alternate universe where twinned worlds ...,2012,6.277,"[Romance, Science Fiction, Drama, Fantasy]"


Additional parameters can be provided for more precise search results. Review the docstring for more information.

In [4]:
conn.hybrid_search??

Signature:
conn.hybrid_search(
    collection_name: str,
    query: str,
    limit: int = 10,
    filters: Optional[weaviate.collections.classes.filters._Filters] = None,
    target_vectors: Union[str, List[str], weaviate.collections.classes.grpc._MultiTargetVectorJoin, NoneType] = None,
    query_properties: Optional[List[str]] = None,
    alpha: float = 0.7,
) -> pandas.core.frame.DataFrame
Source:   
    def hybrid_search(
        self,
        collection_name: str,
        query: str,
        limit: int = 10,
        filters: Optional[_Filters] = None,
        target_vectors: Optional[TargetVectorJoinType] = None,
        query_properties: Optional[List[str]] = None,
        alpha: float = 0.7,
    ) -> pd.DataFrame:
        """
        Query a Weaviate collection using a simplified hybrid query.

        Parameters
        ----------
        collection_name : str
            The name of the collection to query.
        query : str
            The query to search for.
        limit

For example, we can narrow down the results by the release date. Here is the same search, but only for movies from before 2010:

In [5]:
from st_weaviate_connection import WeaviateFilter

df = conn.hybrid_search(
    collection_name="MovieDemo",
    query="Fantasy or sci-fi drama",
    filters=WeaviateFilter.by_property("release_year").less_than(2010)
)

df.head()

,movie_id,vote_count,budget,tagline,title,revenue,overview,release_year,vote_average,genres
0,14337,2098,7000,What happens if it actually works?,Primer,545436,Two fledgling inventors discover a complex met...,2004,6.768,"[Science Fiction, Drama, Thriller]"
1,1124,15086,40000000,Are You Watching Closely?,The Prestige,109676311,A mysterious story of two magicians whose inte...,2006,8.203,"[Drama, Mystery, Science Fiction]"
2,34584,3786,27000000,A boy who needs a friend finds a world that ne...,The NeverEnding Story,20158808,While hiding from bullies in his school's atti...,1984,7.194,"[Adventure, Fantasy, Family, Drama]"
3,25376,2469,2000000,An unsolved crime. A love story. An unwritten ...,The Secret in Their Eyes,33965843,Hoping to put to rest years of unease concerni...,2009,8.002,"[Mystery, Thriller, Drama]"
4,17431,5356,5000000,"250,000 miles from home, the hardest thing to ...",Moon,9760104,With only three weeks left in his three year c...,2009,7.569,"[Science Fiction, Drama]"


If you prefer to use a raw GraphQL query, you can use the `.graphql_query` function.

In [6]:
gql = """
{
  Get {
    MovieDemo (
        limit: 10
      nearText: {
        concepts: ["historical period film"]
      }
    ) {
      title
      overview
      vote_average
      _additional {
        distance
      }
    }
  }
}
"""

df = conn.graphql_query(gql)
df

2024-07-30 21:56:45.618 No runtime found, using MemoryCacheStorageManager
2024-07-30 21:56:45.620 No runtime found, using MemoryCacheStorageManager


,_additional,overview,title,vote_average
0,{'distance': 0.44417703},"A sumptuous and sensual tale of intrigue, roma...",The Other Boleyn Girl,6.685
1,{'distance': 0.47027773},The timeless tale of King Arthur and the legen...,First Knight,6.079
2,{'distance': 0.4740405},"With Ran, legendary director Akira Kurosawa re...",Ran,8.078
3,{'distance': 0.48198485},Akira Kurosawa's lauded feudal epic presents t...,Kagemusha,7.805
4,{'distance': 0.4907261},Katherine Watson is a recent UCLA graduate hir...,Mona Lisa Smile,6.935
5,{'distance': 0.49372083},The retelling of France’s iconic but ill-fated...,Marie Antoinette,6.695
6,{'distance': 0.4953565},"In 25 AD, Judah Ben-Hur, a Jew in ancient Jude...",Ben-Hur,7.889
7,{'distance': 0.49567854},The hot-headed young D'Artagnan along with thr...,The Three Musketeers,5.801
8,{'distance': 0.49683636},"The perversion behind imperial Rome, the epic ...",Caligula,6.016
9,{'distance': 0.4968868},"Alexander, the King of Macedonia, leads his le...",Alexander,5.947


## Advanced Usage

The Streamlit connector is a thin wrapper around the Weaviate Python client. 

This means that you can use the Weaviate Python client directly to perform more advanced operations.

We recommend using the client object in a context manager to ensure that no resources are leaked.

### Example: Perform retrieval augmented generation

In [11]:
with conn.client() as client:
    collection = client.collections.get("MovieDemo")
    response = collection.generate.hybrid(
        limit=20,
        query="Fantasy or sci-fi drama",
        grouped_task="From these, recommend one or two movies that would be family friendly!",
        grouped_properties=["title", "tagline"],
    )

    print("## Generated recommendation")
    print(response.generated)
    print("\n## Source data")
    for o in response.objects:
        print(f"Title: {o.properties['title']}")

## Generated recommendation
Based on the list provided, here are two movies that would be suitable and enjoyable for a family movie night:

**"The NeverEnding Story"**
With the tagline "A boy who needs a friend finds a world that needs a hero in a land beyond imagination!", this classic fantasy film takes viewers on an adventure through a magical world filled with dragons, mysterious creatures, and exciting challenges. It's a story about the power of imagination, courage, and friendship, making it a perfect choice for a family movie night.

**"Jack the Giant Slayer**"
"Prepare for a giant adventure" with this action-packed and imaginative take on the classic fairy tale. The movie follows a young man who opens a gateway to a world of giants, leading to a dangerous and thrilling journey. While there is some fantasy violence, it's generally mild and suitable for older children, making it a fun and exciting choice for families.

Both of these movies offer a great blend of adventure, imagin

### Use a local Weaviate instance

To connect to a local instance instead, specify the `url` parameter as `"localhost"`, and the connector will connect with default settings.

In [ ]:
conn = st.connection(
    "weaviate",
    type=WeaviateConnection,
    url="localhost",
)

## Using Secrets

The following are valid values you can specify in your `secrets.toml` file when using this connection:

| Config | Description |
| --- | --- |
| WEAVIATE_URL | The url of the weavite instance you want to connect to |
| WEAVIATE_API_KEY | The corresponding api key of the weaviate instance you want to connect to (if applicable) |

For more details, refer to:

* [How to use secrets management](https://docs.streamlit.io/library/advanced-features/secrets-management#how-to-use-secrets-management)
* [st.connection](https://docs.streamlit.io/library/api-reference/connections/st.connection)

## Using the Weaviate client library directly

You can also use the Weaviate client library directly to perform more advanced operations.

Please see the [Weaviate Python client documentation](https://weaviate.io/developers/weaviate/client-libraries/python) for more information.